In [1]:
from IPython.core.display import display, HTML
#makes the display bars longer and spread out 100% across the screen width
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

import collections

import warnings
warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

import db_queries as db
import vivarium_helpers.id_helper as idh

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vop

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/model_validation/model2
ndbs
Wed Aug  4 14:50:46 PDT 2021


In [29]:
%load_ext autoreload
%autoreload 2

In [3]:
0o777

511

In [4]:
0xFFF

4095

In [6]:
# /ihme/costeffectiveness/results/vivarium_ciff_sam/v2.5_stunting/ciff_sam/2021_08_05_16_17_12/count_data
results_path = '/ihme/costeffectiveness/results/vivarium_ciff_sam/v2.5_stunting/ciff_sam/2021_08_05_16_17_12/count_data'

In [7]:
data = vto.VivariumTransformedOutput.from_directory(results_path)
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [9]:
data.disease_state_person_time

,sex,year,cause,measure,input_draw,scenario,value,stunting_state,age
0,female,2022,diarrheal_diseases,state_person_time,29,baseline,43.489391,cat4,early_neonatal
1,female,2022,diarrheal_diseases,state_person_time,29,baseline,0.000000,cat3,early_neonatal
2,female,2022,diarrheal_diseases,state_person_time,29,baseline,0.000000,cat2,early_neonatal
3,female,2022,diarrheal_diseases,state_person_time,29,baseline,0.000000,cat1,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,susceptible_to_measles,state_person_time,946,baseline,121118.023272,cat4,2_to_4
17277,male,2026,susceptible_to_measles,state_person_time,946,baseline,67325.987680,cat3,2_to_4
17278,male,2026,susceptible_to_measles,state_person_time,946,baseline,66608.265572,cat2,2_to_4
17279,male,2026,susceptible_to_measles,state_person_time,946,baseline,63010.346338,cat1,2_to_4


In [11]:
data.disease_state_person_time.index.name is None

True

In [12]:
# Ok cool, checking if None is in the columns doesn't throw an error
data.disease_state_person_time.index.name in data.disease_state_person_time

False

In [13]:
None in data.disease_state_person_time

False

In [14]:
None in ['a', 'b', 'c']

False

In [18]:
[] in data.disease_state_person_time

TypeError: unhashable type: 'list'

In [8]:
data.wasting_state_person_time

,sex,year,cause,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,414.090349,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1234.535250,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,8926.954141,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,10434.387406,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,30258.041068,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,35695.241615,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,70879.949350,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,215444.187543,2_to_4


In [15]:
data.wasting_state_person_time.rename(columns={'cause':'wasting_state'}, inplace=True)
data.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,414.090349,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1234.535250,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,8926.954141,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,10434.387406,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,30258.041068,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,35695.241615,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,70879.949350,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,215444.187543,2_to_4


# Write function to get total person time

In [30]:
def get_total_person_time(data):
    return vop.marginalize(data.wasting_state_person_time, 'wasting_state').assign(measure='person_time')
person_time = get_total_person_time(data)
person_time

,age,input_draw,measure,scenario,sex,year,value
0,1-5_months,29,person_time,baseline,female,2022,44140.585900
1,1-5_months,29,person_time,baseline,female,2023,43843.549624
2,1-5_months,29,person_time,baseline,female,2024,43946.220397
3,1-5_months,29,person_time,baseline,female,2025,43707.561944
...,...,...,...,...,...,...,...
716,late_neonatal,946,person_time,baseline,male,2023,6263.611225
717,late_neonatal,946,person_time,baseline,male,2024,6284.217659
718,late_neonatal,946,person_time,baseline,male,2025,6291.129363
719,late_neonatal,946,person_time,baseline,male,2026,6238.913073


In [31]:
vop.marginalize(person_time.set_index('age'), 'age')

,input_draw,measure,scenario,sex,year,value
0,29,person_time,baseline,female,2022,491926.453114
1,29,person_time,baseline,female,2023,499655.611225
2,29,person_time,baseline,female,2024,506966.892539
3,29,person_time,baseline,female,2025,509385.232033
...,...,...,...,...,...,...
116,946,person_time,baseline,male,2023,521780.900753
117,946,person_time,baseline,male,2024,529583.511294
118,946,person_time,baseline,male,2025,532527.890486
119,946,person_time,baseline,male,2026,532910.529774


In [32]:
vop.marginalize(person_time.set_index(['age', 'sex']), 'age')

,input_draw,measure,scenario,sex,year,value
0,29,person_time,baseline,female,2022,491926.453114
1,29,person_time,baseline,female,2023,499655.611225
2,29,person_time,baseline,female,2024,506966.892539
3,29,person_time,baseline,female,2025,509385.232033
...,...,...,...,...,...,...
116,946,person_time,baseline,male,2023,521780.900753
117,946,person_time,baseline,male,2024,529583.511294
118,946,person_time,baseline,male,2025,532527.890486
119,946,person_time,baseline,male,2026,532910.529774


# Test ratio function

In [33]:
data.ylls

,sex,year,cause,measure,input_draw,scenario,value,stunting_state,age
0,female,2022,diarrheal_diseases,ylls,29,baseline,2843.691511,cat4,early_neonatal
1,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,cat3,early_neonatal
2,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,cat2,early_neonatal
3,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,cat1,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,severe_acute_malnutrition,ylls,946,baseline,258.045887,cat4,2_to_4
17277,male,2026,severe_acute_malnutrition,ylls,946,baseline,0.000000,cat3,2_to_4
17278,male,2026,severe_acute_malnutrition,ylls,946,baseline,85.628179,cat2,2_to_4
17279,male,2026,severe_acute_malnutrition,ylls,946,baseline,86.997977,cat1,2_to_4


In [34]:
vop.ratio(data.ylls, person_time, strata=['year', 'sex', 'age'], multiplier=100_000)

,year,sex,age,input_draw,scenario,value
0,2022,female,1-5_months,29,baseline,9.777698e+04
1,2022,female,1-5_months,223,baseline,1.084875e+05
2,2022,female,1-5_months,232,baseline,1.229208e+05
3,2022,female,1-5_months,357,baseline,1.161435e+05
...,...,...,...,...,...,...
716,2026,male,late_neonatal,650,baseline,1.112877e+06
717,2026,male,late_neonatal,680,baseline,7.709513e+05
718,2026,male,late_neonatal,829,baseline,6.533410e+05
719,2026,male,late_neonatal,946,baseline,9.725049e+05


In [35]:
vop.ratio(data.ylls, person_time, strata=['year', 'sex', 'age'], multiplier=100_000, numerator_broadcast=['cause'])

,year,sex,age,input_draw,scenario,cause,value
0,2022,female,1-5_months,29,baseline,diarrheal_diseases,44973.667775
1,2022,female,1-5_months,29,baseline,lower_respiratory_infections,16262.581298
2,2022,female,1-5_months,29,baseline,measles,601.764343
3,2022,female,1-5_months,29,baseline,moderate_acute_malnutrition,1808.135751
...,...,...,...,...,...,...,...
4316,2026,male,late_neonatal,946,baseline,measles,0.000000
4317,2026,male,late_neonatal,946,baseline,moderate_acute_malnutrition,0.000000
4318,2026,male,late_neonatal,946,baseline,other_causes,801644.068244
4319,2026,male,late_neonatal,946,baseline,severe_acute_malnutrition,0.000000


# Test stratify function

In [37]:
vop.stratify(data.ylls, ['year', 'age', 'sex'])

,year,age,sex,input_draw,scenario,value
0,2022,1-5_months,female,29,baseline,43159.333610
1,2022,1-5_months,female,223,baseline,47503.756705
2,2022,1-5_months,female,232,baseline,53703.492723
3,2022,1-5_months,female,357,baseline,50696.853532
...,...,...,...,...,...,...
716,2026,late_neonatal,male,650,baseline,69645.285146
717,2026,late_neonatal,male,680,baseline,48414.949281
718,2026,late_neonatal,male,829,baseline,41219.035784
719,2026,late_neonatal,male,946,baseline,60673.737513


In [38]:
data.ylls.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'moderate_acute_malnutrition', 'other_causes',
       'severe_acute_malnutrition'], dtype=object)

In [44]:
vop.stratify(data.ylls, 'stunting_state')

,stunting_state,input_draw,scenario,value
0,cat1,29,baseline,4.276886e+05
1,cat1,223,baseline,3.890922e+05
2,cat1,232,baseline,5.120414e+05
3,cat1,357,baseline,4.614156e+05
...,...,...,...,...
44,cat4,650,baseline,3.748226e+06
45,cat4,680,baseline,2.892245e+06
46,cat4,829,baseline,2.436488e+06
47,cat4,946,baseline,3.522734e+06


In [41]:
vop.stratify(data.ylls, ['year', 'age', 'cause'], reset_index=False)

value
year age           cause                     input_draw scenario              
2022 1-5_months    diarrheal_diseases        29         baseline  33145.547727
                                             223        baseline  21358.846427
                                             232        baseline  33588.565270
                                             357        baseline  22246.746277
...                                                                        ...
2026 late_neonatal severe_acute_malnutrition 650        baseline      0.000000
                                             680        baseline      0.000000
                                             829        baseline      0.000000
                                             946        baseline      0.000000

[2160 rows x 1 columns]

In [42]:
vop.stratify(data.ylls, ['year', 'age'], reset_index=False).squeeze()

year  age            input_draw  scenario
2022  1-5_months     29          baseline     94563.563485
                     223         baseline    100769.268488
                     232         baseline    115205.003933
                     357         baseline    110965.259650
                                                 ...      
2026  late_neonatal  650         baseline    122856.781034
                     680         baseline     88035.632885
                     829         baseline     75330.873116
                     946         baseline    114151.605098
Name: value, Length: 360, dtype: float64

In [43]:
# Check that stratify still works when strata are in the index
df = vop.stratify(data.ylls, ['year', 'age', 'cause'], reset_index=False)
vop.stratify(df, ['year', 'age'])

,year,age,input_draw,scenario,value
0,2022,1-5_months,29,baseline,94563.563485
1,2022,1-5_months,223,baseline,100769.268488
2,2022,1-5_months,232,baseline,115205.003933
3,2022,1-5_months,357,baseline,110965.259650
...,...,...,...,...,...
356,2026,late_neonatal,650,baseline,122856.781034
357,2026,late_neonatal,680,baseline,88035.632885
358,2026,late_neonatal,829,baseline,75330.873116
359,2026,late_neonatal,946,baseline,114151.605098


# Test difference function

In [49]:
data.ylls

,sex,year,cause,measure,input_draw,scenario,value,stunting_state,age
0,female,2022,diarrheal_diseases,ylls,29,baseline,2843.691511,cat4,early_neonatal
1,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,cat3,early_neonatal
2,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,cat2,early_neonatal
3,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,cat1,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,severe_acute_malnutrition,ylls,946,baseline,258.045887,cat4,2_to_4
17277,male,2026,severe_acute_malnutrition,ylls,946,baseline,0.000000,cat3,2_to_4
17278,male,2026,severe_acute_malnutrition,ylls,946,baseline,85.628179,cat2,2_to_4
17279,male,2026,severe_acute_malnutrition,ylls,946,baseline,86.997977,cat1,2_to_4


In [53]:
vop.difference(data.ylls, 'cause', 'diarrheal_diseases')

cause
['sex', 'year', 'measure', 'input_draw', 'scenario', 'stunting_state', 'age']
['sex', 'year', 'measure', 'input_draw', 'scenario', 'stunting_state', 'age', 'cause']
Index(['sex', 'year', 'measure', 'input_draw', 'scenario', 'stunting_state',
       'age', 'cause', 'value'],
      dtype='object')


,sex,year,measure,input_draw,scenario,stunting_state,age,cause,subtracted_from,value
0,female,2022,ylls,29,baseline,cat1,1-5_months,lower_respiratory_infections,diarrheal_diseases,797.093481
1,female,2022,ylls,29,baseline,cat1,1-5_months,measles,diarrheal_diseases,1240.372439
2,female,2022,ylls,29,baseline,cat1,1-5_months,moderate_acute_malnutrition,diarrheal_diseases,1328.834784
3,female,2022,ylls,29,baseline,cat1,1-5_months,other_causes,diarrheal_diseases,443.087254
...,...,...,...,...,...,...,...,...,...,...
14396,male,2026,ylls,946,baseline,cat4,late_neonatal,measles,diarrheal_diseases,7550.721398
14397,male,2026,ylls,946,baseline,cat4,late_neonatal,moderate_acute_malnutrition,diarrheal_diseases,7550.721398
14398,male,2026,ylls,946,baseline,cat4,late_neonatal,other_causes,diarrheal_diseases,-42463.155177
14399,male,2026,ylls,946,baseline,cat4,late_neonatal,severe_acute_malnutrition,diarrheal_diseases,7550.721398


In [54]:
vop.difference(data.ylls, 'cause', 'diarrheal_diseases', 'measles')

cause
['sex', 'year', 'measure', 'input_draw', 'scenario', 'stunting_state', 'age']
['sex', 'year', 'measure', 'input_draw', 'scenario', 'stunting_state', 'age', 'cause']
Index(['sex', 'year', 'measure', 'input_draw', 'scenario', 'stunting_state',
       'age', 'cause', 'value'],
      dtype='object')


,sex,year,measure,input_draw,scenario,stunting_state,age,cause,subtracted_from,value
0,female,2022,ylls,29,baseline,cat4,early_neonatal,measles,diarrheal_diseases,2843.691511
1,female,2022,ylls,29,baseline,cat3,early_neonatal,measles,diarrheal_diseases,0.000000
2,female,2022,ylls,29,baseline,cat2,early_neonatal,measles,diarrheal_diseases,0.000000
3,female,2022,ylls,29,baseline,cat1,early_neonatal,measles,diarrheal_diseases,0.000000
...,...,...,...,...,...,...,...,...,...,...
2876,male,2026,ylls,946,baseline,cat4,2_to_4,measles,diarrheal_diseases,3684.900169
2877,male,2026,ylls,946,baseline,cat3,2_to_4,measles,diarrheal_diseases,3332.967609
2878,male,2026,ylls,946,baseline,cat2,2_to_4,measles,diarrheal_diseases,4278.292144
2879,male,2026,ylls,946,baseline,cat1,2_to_4,measles,diarrheal_diseases,4871.217593


In [55]:
vop.difference(data.ylls, 'cause', None, 'measles')

['sex', 'year', 'measure', 'input_draw', 'scenario', 'stunting_state', 'age', 'cause']
['sex', 'year', 'measure', 'input_draw', 'scenario', 'stunting_state', 'age']
Index(['sex', 'year', 'measure', 'input_draw', 'scenario', 'stunting_state',
       'age', 'cause', 'value'],
      dtype='object')


,sex,year,measure,input_draw,scenario,stunting_state,age,cause,subtracted_value,value
0,female,2022,ylls,29,baseline,cat1,1-5_months,diarrheal_diseases,measles,1240.372439
1,female,2022,ylls,29,baseline,cat1,1-5_months,lower_respiratory_infections,measles,443.278959
2,female,2022,ylls,29,baseline,cat1,1-5_months,moderate_acute_malnutrition,measles,-88.462344
3,female,2022,ylls,29,baseline,cat1,1-5_months,other_causes,measles,797.285185
...,...,...,...,...,...,...,...,...,...,...
14396,male,2026,ylls,946,baseline,cat4,late_neonatal,lower_respiratory_infections,measles,3109.139541
14397,male,2026,ylls,946,baseline,cat4,late_neonatal,moderate_acute_malnutrition,measles,0.000000
14398,male,2026,ylls,946,baseline,cat4,late_neonatal,other_causes,measles,50013.876575
14399,male,2026,ylls,946,baseline,cat4,late_neonatal,severe_acute_malnutrition,measles,0.000000


In [56]:
[*'ablsalkfjfj']

['a', 'b', 'l', 's', 'a', 'l', 'k', 'f', 'j', 'f', 'j']